In [ ]:
import cv2

In [ ]:
import numpy as np

In [ ]:
import dlib
# dilb is a library in which some machine learning functions  are implemented already for image processing. Dlib is a general purpose cross-platform software library written in the programming language C++. Its design is heavily influenced by ideas from design by contract and component-based software engineering. Thus it is, first and foremost, a set of independent software components. 

In [ ]:
#face_utils for basic operations of conversion
from imutils import face_utils

In [ ]:
#Initializing the camera and taking the instance
cap = cv2.VideoCapture(0)

In [ ]:
#Initializing the face detector and landmark detector
detector = dlib.get_frontal_face_detector()
#get_frontal_face_detector() is inbuilt fucntion of dlib library... it is more accurate than open cv harcascade detector

In [ ]:
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [ ]:
#status marking for current state
sleep = 0
drowsy = 0
active = 0
status=""
color=(0,0,0)

In [ ]:
def compute(ptA,ptB):
	dist = np.linalg.norm(ptA - ptB)
	return dist

In [ ]:
def blinked(a,b,c,d,e,f):
	up = compute(b,d) + compute(c,e)
	down = compute(a,f)
	ratio = up/(2.0*down)

	#Checking if it is blinked
	if(ratio>0.25):
		return 2
	elif(ratio>0.21 and ratio<=0.25):
		return 1
	else:
		return 0

In [ ]:
# pip install pytesseract


In [ ]:
import pytesseract

In [ ]:
# Path to Tesseract executable (optional, if not in your PATH)
# pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

# Function to perform OCR on a frame
def perform_ocr(frame):
    # Convert the frame to grayscale (required for better OCR accuracy)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Perform OCR using pytesseract
    text = pytesseract.image_to_string(gray)

    return text

In [ ]:
# Check if the capture object is opened successfully
if not cap.isOpened():
    print("Error: Failed to open video capture.")
    exit()

In [ ]:
import cv2
import pytesseract
from imutils import face_utils
import dlib

# Path to Tesseract executable (optional, if not in your PATH)
# pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

# Initialize video capture object (0 represents the default webcam)
cap = cv2.VideoCapture(0)

# Load pre-trained face detector and facial landmarks predictor from dlib
detector = dlib.get_frontal_face_detector()
predictor_path = "shape_predictor_68_face_landmarks.dat"
predictor = dlib.shape_predictor(predictor_path)

# Define a function to check for blinks
def blinked(eye_points):
    left_eye_aspect_ratio = calculate_eye_aspect_ratio(eye_points)
    return left_eye_aspect_ratio < 0.25  # Adjust threshold as needed

# Function to calculate the eye aspect ratio (EAR)
def calculate_eye_aspect_ratio(eye_points):
    # Calculate EAR based on eye points
    # This is just a placeholder, you can adjust this function
    return 0.3

# Define a function to perform OCR on a frame
def perform_ocr(frame):
    # Convert the frame to grayscale for better OCR accuracy
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Perform OCR using pytesseract on the grayscale frame
    text = pytesseract.image_to_string(gray)

    return text

# Continuously read frames from the video capture object
while True:
    # Read frame from the video capture
    ret, frame = cap.read()
    
    # Check if the frame is read successfully
    if not ret:
        print("Error: Failed to read frame.")
        break

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Initialize face_frame for drawing on the face
    face_frame = None
    
    # Detect faces in the grayscale frame
    faces = detector(gray)
    for face in faces:
        x1 = face.left()
        y1 = face.top()
        x2 = face.right()
        y2 = face.bottom()

        # Copy the frame and draw rectangles around detected faces
        face_frame = frame.copy()
        cv2.rectangle(face_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Get facial landmarks
        landmarks = predictor(gray, face)
        landmarks = face_utils.shape_to_np(landmarks)

        # Process eye blinks
        left_blink = blinked(landmarks[36:42])
        right_blink = blinked(landmarks[42:48])

        # Initialize status variables
        sleep = 0
        drowsy = 0
        active = 0
        status = ""
        color = (0, 0, 0)

        # Check for eye blinks
        if left_blink or right_blink:
            sleep += 1
            drowsy = 0
            active = 0
            if sleep > 6:
                status = "!!! SLEEPING !!! Alert (Alarm sound on)"
                color = (255, 0, 0)
        else:
            sleep = 0
            active += 1
            drowsy = 0
            if active > 6:
                status = "Active :)"
                color = (0, 255, 0)

        # Draw status text on the frame
        cv2.putText(frame, status, (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.2, color, 3)

        # Draw facial landmarks on the face frame
        for n in range(0, 68):
            (x, y) = landmarks[n]
            cv2.circle(face_frame, (x, y), 1, (255, 255, 255), -1)

    # Perform OCR on the frame
    extracted_text = perform_ocr(frame)

    # Print the extracted text in the console
    print("Extracted Text:", extracted_text)

    # Show the original frame
    cv2.imshow("Frame", frame)

    # Show the face frame if there were detected faces
    if face_frame is not None:
        cv2.imshow("Face Frame", face_frame)

    # Define a region of interest (ROI)
    roi = frame[50:350, 50:350]

    # Show the ROI
    cv2.imshow("ROI", roi)

    # Check for 'q' key to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close OpenCV windows
cap.release()
cv2.destroyAllWindows()


In [ ]:
while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    face_frame = None 
    
    # Check if frame is read successfully
    if not ret:
        print("Error: Failed to read frame.")
        break

    faces = detector(gray)#detected face in faces array
    for face in faces:
        x1 = face.left()
        y1 = face.top()
        x2 = face.right()
        y2 = face.bottom()

        face_frame = frame.copy()
        cv2.rectangle(face_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        landmarks = predictor(gray, face)
        landmarks = face_utils.shape_to_np(landmarks)

        #The numbers are actually the landmarks which will show eye
        left_blink = blinked(landmarks[36],landmarks[37],landmarks[38], landmarks[41], landmarks[40], landmarks[39])
        right_blink = blinked(landmarks[42],landmarks[43],landmarks[44], landmarks[47], landmarks[46], landmarks[45])

        #Now judge what to do for the eye blinks
        if(left_blink==0 or right_blink==0):
            sleep+=1
            drowsy=0
            active=0
            if(sleep>6):
                status="!!! SLEEPING !!! Alert (Alarm sound on) "
                color = (255,0,0)

        elif(left_blink==1 or right_blink==1):
            sleep=0
            active=0
            drowsy+=1
            if(drowsy>6):
                status="Drowsy !"
                color = (0,0,255)

        else:
            drowsy=0
            sleep=0
            active+=1
            if(active>6):
                status="Active :)"
                color = (0,255,0)

        cv2.putText(frame, status, (100,100), cv2.FONT_HERSHEY_SIMPLEX, 1.2, color,3)

        for n in range(0, 68):
            (x,y) = landmarks[n]
            cv2.circle(face_frame, (x, y), 1, (255, 255, 255), -1)

    # Access pixel value at specific coordinates
    pixel_value = frame[100, 100]
    print("Pixel value at (100, 100):", pixel_value)

    # Access color channels
    blue_channel = frame[:, :, 0]
    green_channel = frame[:, :, 1]
    red_channel = frame[:, :, 2]

    # Display the shape of the frame and channels
    print("Frame shape:", frame.shape)
    print("Blue channel shape:", blue_channel.shape)
    print("Green channel shape:", green_channel.shape)
    print("Red channel shape:", red_channel.shape)

    # Define a region of interest (ROI) and extract data
    # roi = frame[50:350, 50:350]
    # cv2.imshow("ROI", roi)
    
    cv2.imshow("Frame", frame)
    # cv2.imshow("Frame", face_frame)
    # cv2.imshow("Result of detector", face_frame)
    # key = cv2.waitKey(1) 
    # if key == 27:
    #   	break

    # Perform OCR on the frame
    text = perform_ocr(frame)
    # Print the extracted text in the console
    print("Extracted Text:", text)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
# while True:
#     _, frame = cap.read()
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     face_frame = None 
    
#     faces = detector(gray)#detected face in faces array
#     for face in faces:
#         x1 = face.left()
#         y1 = face.top()
#         x2 = face.right()
#         y2 = face.bottom()

#         face_frame = frame.copy()
#         cv2.rectangle(face_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

#         landmarks = predictor(gray, face)
#         landmarks = face_utils.shape_to_np(landmarks)

#         #The numbers are actually the landmarks which will show eye
#         left_blink = blinked(landmarks[36],landmarks[37],landmarks[38], landmarks[41], landmarks[40], landmarks[39])
#         right_blink = blinked(landmarks[42],landmarks[43],landmarks[44], landmarks[47], landmarks[46], landmarks[45])

#         #Now judge what to do for the eye blinks
#         if(left_blink==0 or right_blink==0):
#             sleep+=1
#             drowsy=0
#             active=0
#             if(sleep>6):
#                 status="!!! SLEEPING !!! Alert (Alarm sound on) "
#                 color = (255,0,0)

#         elif(left_blink==1 or right_blink==1):
#             sleep=0
#             active=0
#             drowsy+=1
#             if(drowsy>6):
#                 status="Drowsy !"
#                 color = (0,0,255)

#         else:
#             drowsy=0
#             sleep=0
#             active+=1
#             if(active>6):
#                 status="Active :)"
#                 color = (0,255,0)

#         cv2.putText(frame, status, (100,100), cv2.FONT_HERSHEY_SIMPLEX, 1.2, color,3)

#         for n in range(0, 68):
#             (x,y) = landmarks[n]
#             cv2.circle(face_frame, (x, y), 1, (255, 255, 255), -1)

#     # Access pixel value at specific coordinates
#     pixel_value = frame[100, 100]
#     print("Pixel value at (100, 100):", pixel_value)

#     # Access color channels
#     blue_channel = frame[:, :, 0]
#     green_channel = frame[:, :, 1]
#     red_channel = frame[:, :, 2]

#     # Display the shape of the frame and channels
#     print("Frame shape:", frame.shape)
#     print("Blue channel shape:", blue_channel.shape)
#     print("Green channel shape:", green_channel.shape)
#     print("Red channel shape:", red_channel.shape)

#     # Define a region of interest (ROI) and extract data
#     roi = frame[50:350, 50:350]
#     cv2.imshow("ROI", roi)
    
#     cv2.imshow("Frame", frame)
#     # cv2.imshow("Frame", face_frame)
#     # cv2.imshow("Result of detector", face_frame)
#     # key = cv2.waitKey(1) 
#     # if key == 27:
#     #   	break
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()

In [ ]:
# while True:
#     _, frame = cap.read()
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     face_frame = None 
    
#     faces = detector(gray)#detected face in faces array
#     for face in faces:
#         x1 = face.left()
#         y1 = face.top()
#         x2 = face.right()
#         y2 = face.bottom()

#         face_frame = frame.copy()
#         cv2.rectangle(face_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

#         landmarks = predictor(gray, face)
#         landmarks = face_utils.shape_to_np(landmarks)

#         #The numbers are actually the landmarks which will show eye
#         left_blink = blinked(landmarks[36],landmarks[37],landmarks[38], landmarks[41], landmarks[40], landmarks[39])
#         right_blink = blinked(landmarks[42],landmarks[43],landmarks[44], landmarks[47], landmarks[46], landmarks[45])

#         #Now judge what to do for the eye blinks
#         if(left_blink==0 or right_blink==0):
#             sleep+=1
#             drowsy=0
#             active=0
#             if(sleep>6):
#                 status="!!! SLEEPING !!! Alert (Alarm sound on) "
#                 color = (255,0,0)

#         elif(left_blink==1 or right_blink==1):
#             sleep=0
#             active=0
#             drowsy+=1
#             if(drowsy>6):
#                 status="Drowsy !"
#                 color = (0,0,255)

#         else:
#             drowsy=0
#             sleep=0
#             active+=1
#             if(active>6):
#                 status="Active :)"
#                 color = (0,255,0)

#         cv2.putText(frame, status, (100,100), cv2.FONT_HERSHEY_SIMPLEX, 1.2, color,3)

#         for n in range(0, 68):
#             (x,y) = landmarks[n]
#             cv2.circle(face_frame, (x, y), 1, (255, 255, 255), -1)

#     # Access pixel value at specific coordinates
#     pixel_value = frame[100, 100]
#     print("Pixel value at (100, 100):", pixel_value)

#     # Access color channels
#     blue_channel = frame[:, :, 0]
#     green_channel = frame[:, :, 1]
#     red_channel = frame[:, :, 2]

#     # Display the shape of the frame and channels
#     print("Frame shape:", frame.shape)
#     print("Blue channel shape:", blue_channel.shape)
#     print("Green channel shape:", green_channel.shape)
#     print("Red channel shape:", red_channel.shape)

#     # Define a region of interest (ROI) and extract data
#     roi = frame[50:350, 50:350]
#     cv2.imshow("ROI", roi)
    
#     cv2.imshow("Frame", frame)
#     # cv2.imshow("Frame", face_frame)
#     # cv2.imshow("Result of detector", face_frame)
#     # key = cv2.waitKey(1) 
#     # if key == 27:
#     #   	break
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()